In [1]:
import os
from pathlib import Path
from dotenv import load_dotenv
import requests
import pandas as pd
import gspread
from datetime import datetime
from gspread_dataframe import set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials

# === Descobre o diretório do projeto dinamicamente ===
try:
    # Tentativa para ambientes que suportam __file__ (scripts)
    base_dir = Path(__file__).resolve().parent
except NameError:
    # Fallback para notebooks (Jupyter, Papermill)
    base_dir = Path().resolve()

# === Carrega o .env da pasta /secrets relativa ao projeto ===
dotenv_path = base_dir.parent / "secrets" / ".env"
load_dotenv(dotenv_path, override=True)

# === Carrega variáveis do ambiente ===
API_URL = os.getenv("API_URL")
API_KEY = os.getenv("API_KEY")
HEADERS = {"Api-Token": API_KEY}

# === Testa conexão com a API ===
def testar_conexao():
    try:
        url = f"{API_URL}/contacts?limit=1"
        response = requests.get(url, headers=HEADERS)
        response.raise_for_status()
        print("✅ Conexão com a API bem-sucedida.")
        contato_teste = response.json().get("contacts", [{}])[0]
        print("Exemplo de contato:", contato_teste.get("email", "sem email"))
    except requests.exceptions.RequestException as e:
        print("❌ Falha na conexão com a API:", e)
        print("🔎 URL testada:", url)
    except Exception as e:
        print("❌ Erro inesperado:", e)

testar_conexao()

✅ Conexão com a API bem-sucedida.
Exemplo de contato: 022998828178lucianodahira@gmail.com


In [2]:
requests.get(f"{API_URL}/fields", headers=HEADERS)

<Response [200]>

In [3]:
# Buscar todos os campos personalizados
response = requests.get(f"{API_URL}/fields", headers=HEADERS)
response.raise_for_status()

campos = response.json().get("fields", [])

print("\n=== Campos Personalizados ===")
for campo in campos:
    print(f"ID: {campo['id']} - Nome: {campo['title']}")


=== Campos Personalizados ===
ID: 6 - Nome: estado
ID: 7 - Nome: idade
ID: 8 - Nome: escolaridade
ID: 9 - Nome: renda
ID: 10 - Nome: estado civil
ID: 11 - Nome: filhos
ID: 12 - Nome: escolheu profissão
ID: 13 - Nome: dificuldade
ID: 14 - Nome: email captação
ID: 15 - Nome: telefone captação
ID: 18 - Nome: data inscrição lançamento
ID: 24 - Nome: utm_source
ID: 25 - Nome: utm_campaign
ID: 26 - Nome: utm_medium
ID: 27 - Nome: utm_content
ID: 28 - Nome: utm_term


In [4]:
def buscar_todos_contatos(api_url, headers):
    contatos = []
    field_values = []
    offset = 0
    limit = 100

    while True:
        url = f"{api_url}/contacts?limit={limit}&offset={offset}&include=fieldValues"
        resp = requests.get(url, headers=headers)
        resp.raise_for_status()
        data = resp.json()

        novos_contatos = data.get("contacts", [])
        novos_fields = data.get("fieldValues", [])

        if not novos_contatos:
            break

        contatos.extend(novos_contatos)
        field_values.extend(novos_fields)

        offset += limit

    return contatos, field_values

# Buscar campos personalizados
resp_fields = requests.get(f"{API_URL}/fields", headers=HEADERS)
resp_fields.raise_for_status()
campos_personalizados = {field["id"]: field["title"] for field in resp_fields.json().get("fields", [])}

# Buscar todos os contatos paginando
contatos, field_values = buscar_todos_contatos(API_URL, HEADERS)

# Mapear valores personalizados
campos_por_contato = {}
for fv in field_values:
    contact_id = fv["contact"]
    field_id = fv["field"]
    value = fv["value"]
    titulo = campos_personalizados.get(field_id, f"custom_{field_id}")
    campos_por_contato.setdefault(contact_id, {})[titulo] = value

# Montar DataFrame
dados_contatos = []
for contato in contatos:
    contato_id = contato["id"]
    dados = {
        "id": contato_id,
        "nome": contato.get("firstName", ""),
        "email": contato.get("email", ""),
        "telefone": contato.get("phone", "")
    }
    dados.update(campos_por_contato.get(contato_id, {}))
    dados_contatos.append(dados)

df_leads_l34 = pd.DataFrame(dados_contatos)
df_leads_l34.head()

,id,nome,email,telefone,utm_source,utm_campaign,utm_medium,utm_content,utm_term,data inscrição lançamento,estado,idade,escolaridade,renda,estado civil,filhos,escolheu profissão,dificuldade,email captação,telefone captação
0,9911,Luciano,022998828178lucianodahira@gmail.com,,Facebook-Ads,OPC-L34 | CAPTACAO | CBO | MORNO/QUENTE | BR |...,00 | PAGINA DE CAPTURA 180D,ads_001_captacao_video_stories_p1_imagem_sobral_1,Instagram_Stories,2025-05-18,Outro,46 - 55 anos,Outro,"De R$ 1.000,00 a R$ 3.000,00",Divorciado(a) ou Separado(a),Sim,Gosta da profissão,Financeiro / Dinheiro,022998828178lucianodahira@gmail.com,73981270523
1,9392,Lucas Silva da Costa,71984627341.luca@gmail.com,,Facebook-Ads,OPC-L34 | CAPTACAO | CBO | FRIO | BR | 16-05-2025,00 | LAL 1% LEADS QUALIFICADOS,ads_001_captacao_video_stories_p1_imagem_sobral_1,Instagram_Stories,2025-05-17,Outro,26 - 35 anos,Ensino Fundamental Completo,"De R$ 1.000,00 a R$ 3.000,00",Solteiro(a),Não,Gosta da profissão,Falta de tempo,71984627341.luca@gmail.com,71984627341
2,10061,Alex Santiago,969016114alex@gmail.com,,Facebook-Ads,OPC-L34 | CAPTACAO | CBO | QUENTE | BR | 16-05...,00 | ENVOLVIMENTO 60D,ads_005_captacao_video_stories_p1_video_vemai_v1,Facebook_Mobile_Feed,2025-05-18,São Paulo,36 - 45 anos,Ensino Médio Completo,"De R$ 1.000,00 a R$ 3.000,00",União estável,Sim,Prestígio da carreira,Financeiro / Dinheiro,969016114alex@gmail.com,11950289005
3,9710,Anderson,a.barros.figueiredo@bol.com.br,,Facebook-Ads,OPC-L34 | CAPTACAO | ABO | FRIO LAL10% LISTA A...,00 | LAL 10% LISTA ALUNO | AD04,ads_004_captacao_video_stories_p1_video_4motiv...,Instagram_Feed,2025-05-18,São Paulo,36 - 45 anos,Ensino Médio Completo,"De R$ 1.000,00 a R$ 3.000,00",Solteiro(a),Não,Sonho de criança,Idade,a.barros.figueiredo@bol.com.br,11949807647
4,9684,vitor,a.firma.vitor@gmail.com,,Facebook-Ads,OPC-L34 | CAPTACAO | ABO | FRIO LAL10% LISTA A...,00 | LAL 10% LISTA ALUNO | AD04,ads_004_captacao_video_stories_p1_video_4motiv...,Instagram_Stories,2025-05-18,São Paulo,Até 25 anos,Ensino Médio Completo,"De R$ 1.000,00 a R$ 3.000,00",Solteiro(a),Não,Sonho de criança,Falta de oportunidades na vida,a.firma.vitor@gmail.com,+55 (11) 94038-3044


In [5]:
df_leads_l34.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1686 entries, 0 to 1685
Data columns (total 20 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   id                         1686 non-null   object
 1   nome                       1686 non-null   object
 2   email                      1686 non-null   object
 3   telefone                   1686 non-null   object
 4   utm_source                 1170 non-null   object
 5   utm_campaign               1170 non-null   object
 6   utm_medium                 1170 non-null   object
 7   utm_content                1156 non-null   object
 8   utm_term                   1156 non-null   object
 9   data inscrição lançamento  1686 non-null   object
 10  estado                     1329 non-null   object
 11  idade                      1313 non-null   object
 12  escolaridade               1320 non-null   object
 13  renda                      1323 non-null   object
 14  estado c

In [6]:
def carregar_csv(caminho_arquivo):
    caminho = Path(caminho_arquivo)
    if not caminho.exists():
        print(f"[ERRO] Arquivo CSV não encontrado: {caminho}")
        return None

    for sep in [',', ';', '\t', '|']:
        try:
            df = pd.read_csv(caminho, sep=sep, engine='python')
            print(f"[OK] CSV carregado com separador '{sep}': {caminho.name}")
            return df
        except Exception as e:
            print(f"[INFO] Tentativa com separador '{sep}' falhou: {e}")

    print(f"[ERRO] Falha ao carregar CSV '{caminho.name}' com os separadores testados.")
    return None

# Caminho para o CSV do Elementor
csv_elementor_path = Path.cwd().parent / "dados" / "elementor-submissions.csv"

# Carregamento
df_elementor_leads = carregar_csv(csv_elementor_path)

[OK] CSV carregado com separador ',': elementor-submissions.csv


In [7]:
# Renomeia "E-mail" para "email" no df_elementor
df_elementor_leads.rename(columns={"E-mail": "email"}, inplace=True)

# Normaliza os e-mails para garantir matching
df_leads_l34["email"] = df_leads_l34["email"].str.strip().str.lower()
df_elementor_leads["email"] = df_elementor_leads["email"].str.strip().str.lower()

# Lista das colunas UTM
utms = ["utm_source", "utm_campaign", "utm_medium", "utm_content", "utm_term"]

# Reduz df_elementor_leads a email + utms, removendo duplicados
df_elementor_reduced = df_elementor_leads[["email"] + utms].drop_duplicates("email")

# Faz o merge, deixando pandas adicionar _x e _y
df_leads_l34 = pd.merge(df_leads_l34, df_elementor_reduced, on="email", how="left", suffixes=("_x", "_y"))

# Resolve duplicatas de UTM, preferindo os valores do Elementor (_y)
for utm in utms:
    df_leads_l34[utm] = df_leads_l34[f"{utm}_y"].combine_first(df_leads_l34.get(f"{utm}_x"))

# Remove colunas com sufixos
df_leads_l34.drop(columns=[f"{utm}_x" for utm in utms] + [f"{utm}_y" for utm in utms], inplace=True)

In [8]:
df_elementor_leads.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 516 entries, 0 to 515
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Nome            511 non-null    object 
 1   email           516 non-null    object 
 2   Whatsapp        516 non-null    object 
 3   utm_medium      159 non-null    object 
 4   utm_campaign    159 non-null    object 
 5   utm_source      159 non-null    object 
 6   utm_term        159 non-null    object 
 7   utm_content     159 non-null    object 
 8   Form Name (ID)  516 non-null    object 
 9   Submission ID   516 non-null    int64  
 10  Created At      516 non-null    object 
 11  User ID         516 non-null    int64  
 12  User Agent      0 non-null      float64
 13  User IP         0 non-null      float64
 14  Referrer        516 non-null    object 
dtypes: float64(2), int64(2), object(11)
memory usage: 60.6+ KB


In [9]:
df_leads_l34.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1686 entries, 0 to 1685
Data columns (total 20 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   id                         1686 non-null   object
 1   nome                       1686 non-null   object
 2   email                      1686 non-null   object
 3   telefone                   1686 non-null   object
 4   data inscrição lançamento  1686 non-null   object
 5   estado                     1329 non-null   object
 6   idade                      1313 non-null   object
 7   escolaridade               1320 non-null   object
 8   renda                      1323 non-null   object
 9   estado civil               1317 non-null   object
 10  filhos                     1323 non-null   object
 11  escolheu profissão         1325 non-null   object
 12  dificuldade                1273 non-null   object
 13  email captação             1329 non-null   object
 14  telefone

In [10]:
# Nome do arquivo da planilha que você deseja
NOME_PLANILHA = "Leads L34"
ABA_NOME = "Página1"  # Pode ser alterado se quiser

# Caminho das credenciais
CRED_PATH = Path.cwd().parent / os.getenv("GOOGLE_CREDENTIALS_PATH")

if not CRED_PATH.exists():
    raise FileNotFoundError(f"❌ Credencial não encontrada em: {CRED_PATH}")
else:
    print(f"✅ Credencial localizada: {CRED_PATH}")

# E-mail pessoal (para compartilhar a planilha com você)
EMAIL_PESSOAL = "camilobf2@gmail.com"  # <<< SUBSTITUA pelo seu e-mail real

# Autenticação
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
credenciais = ServiceAccountCredentials.from_json_keyfile_name(str(CRED_PATH), scope)
client = gspread.authorize(credenciais)

# Tenta abrir a planilha, senão cria
try:
    planilha = client.open(NOME_PLANILHA)
except gspread.SpreadsheetNotFound:
    print("📄 Planilha não encontrada. Criando nova...")
    planilha = client.create(NOME_PLANILHA)
    planilha.share(EMAIL_PESSOAL, perm_type='user', role='writer')

# Pega a aba principal
aba = planilha.sheet1

# Limpa conteúdo antigo
aba.clear()

# Escreve DataFrame no Google Sheets
set_with_dataframe(aba, df_leads_l34)

print("✅ Dados enviados com sucesso para o Google Sheets.")

✅ Credencial localizada: C:\Users\Camilo_Bica\data_science\consultoria\escola_policia\secrets\credenciais_gsheets.json
✅ Dados enviados com sucesso para o Google Sheets.


In [11]:
# No notebook de geração:
saida_parquet = Path.cwd().parent / "dados" / "leads_l34.parquet"
df_leads_l34.to_parquet(saida_parquet, index=False)